### Use the Taylor expansion to derive the result of logarithms on dual numbers.

Let $\epsilon^2=0$, the Taylor expansion will become
\begin{align}
 f(a +b\epsilon) & = f(a) + f'(a)b\epsilon 
\end{align}

### Arithmetic rule for dual number

\begin{align}
(a_1 + b_1\epsilon) + (a_2 + b_2\epsilon) & = 
(a_1 + a_2) + (b_1+b_2)\epsilon                      \\
(a_1 + b_1\epsilon) * (a_2 + b_2\epsilon) & = 
a_1a_2 + (a_1b_2+a_2b_1)\epsilon                     \\
c*(a_1+b_1\epsilon) & = ca_1+cb_1\epsilon            \\
\frac{a_1+b_1\epsilon}{a_2+b_2\epsilon} &= 
\frac{a_1}{a_2} +\frac{b_1a_2-a_1b_2}{a_2^2}\epsilon \\
(a+b\epsilon)^n & = a^n + na^{n-1}b\epsilon          \\
e^{a+b\epsilon} & = e^a + e^a b\epsilon              \\
\end{align}

```Julia
import Base: +, -, *, /, log

struct DualNumber{T1, T2} <: Real   
    re::T1                # differnt types so (Float64, Int64), etc., is possible
    du::T2
end

+(x::DualNumber, y::DualNumber) = DualNumber(x.re + y.re, x.du + y.du)  # dual addition
+(x::DualNumber, a::Number) = DualNumber(x.re + a, x.du)  
+(a::Number, x::DualNumber) = DualNumber(x.re + a, x.du) 

-(x::DualNumber, y::DualNumber) = DualNumber(x.re - y.re, x.du - y.du)  # dual subtraction
-(x::DualNumber, a::Number) = DualNumber(x.re - a, x.du)  
-(a::Number, x::DualNumber) = DualNumber(x.re - a, x.du)  

*(x::DualNumber, y::DualNumber) = DualNumber(x.re*y.re, x.re*y.du + y.re*x.du)
*(x::DualNumber, a::Number) = DualNumber(a*x.re, a*x.du)
*(a::Number, x::DualNumber) = DualNumber(a*x.re, a*x.du)

/(x::DualNumber, y::DualNumber) = DualNumber(x.re/y.re, (x.du*y.re-x.re*y.du)/(y.re^2))
/(x::DualNumber, a::Number) = DualNumber(x.re/a, x.du/a)
/(a::Number, x::DualNumber) = DualNumber(a/x.re, (-x.du)*a/x.re)

log(x::DualNumber) = DualNumber(log(x.re), (1/x.re)*x.du)
```

In [3]:
struct DualNumber{T1, T2} <: Number 
    re::T1
    du::T2
end

Base.:+(x::DualNumber, y::DualNumber) = DualNumber(x.re + y.re, x.du + y.du)
Base.:-(x::DualNumber, y::DualNumber) = DualNumber(x.re-y.re, x.du-y.du)
Base.:*(x::DualNumber, y::DualNumber) = DualNumber(x.re*y.re, x.re*y.du + y.re*x.du)
Base.:^(x::DualNumber, n::Union{Float64, Int64}) = DualNumber(x.re^n, n*x.re^(n-1)*x.du)  # cannot use n<:Real, since n is variable
Base.exp(x::DualNumber) = DualNumber(exp(x.re), exp(x.re)*x.du)

In [11]:
"""
our goal is to let the following command work:
promote(DualNumber(2., 3.), 3.)

including two step:
1. which to convert? DualNumber(2., 3.) or 3. ?
2. how to convert?
"""

# 1. which to convert
Base.promote_rule(::Type{DualNumber{T1, T2}}, ::Type{<:Real}) where{T1, T2} = DualNumber{T1, T2}

# 2. how to convert
Base.convert(::Type{DualNumber{T1, T2}}, x::Real) where{T1, T2} = DualNumber(x, zero(float(Int)))

##### Promotion
Promotion refers to converting values of mixed types to a single common type.
```Julia
julia> promote(1, 2.5)
(1.0, 2.5)

# the promote rule in Julia Base
promote_rule(::Type{Float64}, ::Type{Int64}) = Float64
```

##### Conversion
Convert a value from one type to another without the programmer asking for it explicitly.
```Julia
julia> x = 12
12

julia> typeof(x)
Int64

julia> xf = convert(AbstractFloat, x)
12.0

julia> typeof(xf)
Float64

# the convert rule in Julia Base
convert(::Type{T}, x::Number) where {T<:Number} = T(x)
```

###### reference: https://docs.julialang.org/en/v1/manual/conversion-and-promotion/

# Homework

This homework asks you to compute a function's derivatives using symbolic and automatic differentiation methods. Our objective function is the follows.

\begin{align}
 f(x) = \log(x)\times \exp\left[-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2 \right].
\end{align}

Let's assume $\mu=1.5$ and $\sigma=2$. 

In [9]:
f(x; μ=1.5, σ=2.) = log(x) * exp((-1/2) * ((x-μ)/σ)^2);

## Automatic Differentiations

### Use the Taylor expansion to derive the result of logarithms on dual numbers.

\begin{align}
 f(a +b\epsilon) & = f(a) + f'(a)b\epsilon = log(a) + \frac 1a b\epsilon; \\
\Longrightarrow \quad log(a+b\epsilon) & = log(a) + \frac 1a b\epsilon.
\end{align}

### Add methods of the division ("/") and logarithm ("log()") for dual numbers in Julia.

In [4]:
Base.:/(x::DualNumber, y::DualNumber) = DualNumber(x.re/y.re, (x.du*y.re-y.du*x.re)/y.re^2)
Base.log(x::DualNumber) = DualNumber(log(x.re), x.du/x.re)

### Use the dual numbers you've defined to compute the derivative of $f(x)$ at $x=1.2$.

\begin{align}
  f(a+b\epsilon) = f(a) + f'(a) b \epsilon.
\end{align}

Let $b=1$, then

\begin{align}
  f(a+\epsilon) = f(a) + f'(a)  \epsilon.
\end{align}

In [7]:
auto_diff(f, x) = f(DualNumber(x, 1.)).du

auto_diff (generic function with 1 method)

\begin{align}
 f(x) = \log(x)\times \exp\left[-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2 \right].
\end{align}

### Question： What did we do ?

Show the algorithm in $\left(\frac{x-\mu}{\sigma}\right)^2$ part.

**1.**  $v_1 = (x - \mu)$

**2.** $v_2 = \frac{v_1}{\sigma}$

**3.** $v_3 = (v_2)^2$

|  math               |    code    |                         result                        | 
|:--------------------:|:-------------------:|:------------------------------------------------------------------:|
| $$x - \mu$$          | $$(a,1) - (\mu,0)$$                  | $$(a-\mu,\ 1)$$                                   |     
| $$\frac{x}{\sigma}$$ | $$(a-\mu, 1) \div (\sigma, 0)$$      | $$(\frac{a-\mu}{\sigma}, \frac{1  \sigma - (a-\mu) 0}{\sigma^2}) \\ = (\frac{a-\mu}{\sigma}, \frac{1}{\sigma})$$             |   
| $$x^2$$              | $$(\frac{a-\mu}{\sigma}, \frac{1}{\sigma})^2$$ |  $$ ((\frac{a-\mu}{\sigma})^2, 2 (\frac{a-\mu}{\sigma})  \frac{1}{\sigma})$$

In [18]:
using ForwardDiff, SymPy, Symbolics

### Use Julia's package `ForwardDiff`, which implements the forward-mode auto differentiation, to compute the derivative of $f(x)$ at $x=1.2$.

In [16]:
ForwardDiff.derivative(f, 1.2)

0.8375320155347377

## Symbolic Differentiation

### Use Julia's package `SymPy` to compute the analytic derivative of $f(x)$ and evaluate the result at $x=1.2$.
- Hint: You may need to use `diff` and `subs` functions in the package.
- Remark: I intended to use `Symbolics` but find it less straightforward. You might give it a try.

In [17]:
@vars x
diff(f(x), x).subs(x, 1.2)

0.837532015534738

In [21]:
@variables x

symbolic_diff = eval(
     build_function(Symbolics.derivative(f(x), x), x)
)

symbolic_diff(1.2)

0.8375320155347377